# Librairies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data

https://www.data.gouv.fr/fr/datasets/crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012/

The excel file has two types of sheets : national police & local one.

## Understanding one type of sheet : National Police in 2012

In [2]:
#For the national police on 2012
NP_2012=pd.read_excel("crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx",sheet_name=2).rename({"Départements":"crimeType"},axis=1).set_index("crimeType")

In [3]:
NP_2012.head()

,01,01.1,01.2,02,02.1,02.2,02.3,02.4,03,03.1,...,974.1,974.2,974.3,974.4,976,976.1,987,987.1,988,988.1
crimeType,,,,,,,,,,,,,,,,,,,,,
Règlements de compte entre malfaireurs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Homicides pour voler et à l'occasion de vols,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Homicides pour d'autres motifs,0,0,1,0,2,0,0,0,0,0,...,7,0,1,2,0,1,0,0,0,7
Tentatives d'homicides pour voler et à l'occasion de vols,0,0,0,0,0,1,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
Tentatives homicides pour d'autres motifs,0,0,0,0,4,1,0,0,0,1,...,3,0,1,0,0,1,0,0,0,1


We have as columns the sub distrits of France : in our country, we have 95 "départements" which are here subdivided from 2 to 18 subdivisions. We thus have to group by the number of departements to plot it on a map. 

### Overall sum

We will sum for each year all crimes in France.

In [4]:
NP_2012.sum(axis=1)

crimeType
Règlements de compte entre malfaireurs                          34
Homicides pour voler et à l'occasion de vols                    14
Homicides pour d'autres motifs                                 387
Tentatives d'homicides pour voler et à l'occasion de vols       68
Tentatives homicides pour d'autres motifs                      912
                                                             ...  
Infractions à l'exercice d'une profession règlementée          601
Infractions au droit de l'urbanisme et de la construction      833
Fraudes fiscales                                               524
Autres délits économiques et financiers                       9530
Autres délits                                                65181
Length: 107, dtype: int64

### Group by departement

The case of the Corse makes it tricky. All departements are labelled with numbers from 1 to 992, but for this island, we don't have numbers but '2A' and '2B'. 

In [5]:
columns=NP_2012.columns
lenDF=len(columns)

dfSum=pd.DataFrame(index=NP_2012.index)
NP_2012IndexC=NP_2012.copy()

countDep=1 #counter for the departements 
countLines=0

depDf=pd.DataFrame(index=NP_2012.index)

A=False
B=False

We have two specific cases to deal with:
* gap from dep n° 19 to 21,
* gap from 29 to 2A&2B to 31. 

In [6]:
for index, value in enumerate(columns):

    if value=='30': 
        countDep=30

    if value in ['2A','2A.1','2A.2','2A.3','2A.4']: #first case of Corse
        
            newCol=1000+index
            NP_2012[str(newCol)]= NP_2012.iloc[:,index] #adding at the end a column
            
            NP_2012=NP_2012.drop(str(value),axis=1)
            depDf=pd.DataFrame(index=NP_2012.index)

            if columns[index+1]=='2B':
                A=True
        

    elif value in ['2B','2B.1','2B.2','2B.3','2B.4']: #second case of Corse
       
            newCol=2000+index
            NP_2012[str(newCol)]= NP_2012.iloc[:,index] #adding at the end a column
            NP_2012=NP_2012.drop(str(value),axis=1)
            depDf=pd.DataFrame(index=NP_2012.index)

            if columns[index+1]=='30':
                B=True

    
    elif ( countDep<=float(value) )  and ( float(value) < countDep+1 ) :
        
        depDf[str(index)]=NP_2012.iloc[:,index]

        try:
            if float(columns[index+1])==21 and countDep+1==20:
                countDep+=1
        except:
            continue

        if columns[index+1]=='2A':

            countDep=29
            dfSum[str(countDep)]=depDf.sum(axis=1)  
            depDf=pd.DataFrame(index=NP_2012.index)

        if columns[index+1]!='2A':
            if float(columns[index+1])==countDep+1 or float(columns[index+1])==971 and float(value)!=1 :
                
                dfSum[str(countDep)]=depDf.sum(axis=1) 
                countDep+=1
                depDf=pd.DataFrame(index=NP_2012.index)

In [7]:
dfSum

,1,2,3,4,5,6,7,8,9,10,...,86,87,88,89,90,91,92,93,94,95
crimeType,,,,,,,,,,,,,,,,,,,,,
Règlements de compte entre malfaireurs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,3,0,0
Homicides pour voler et à l'occasion de vols,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Homicides pour d'autres motifs,1,2,2,0,0,22,0,0,0,0,...,1,0,1,0,0,4,15,16,4,16
Tentatives d'homicides pour voler et à l'occasion de vols,0,1,0,0,0,5,0,0,0,0,...,0,0,0,0,0,0,8,4,1,0
Tentatives homicides pour d'autres motifs,0,5,2,0,0,12,0,0,0,0,...,2,0,1,0,3,39,44,64,36,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Infractions à l'exercice d'une profession règlementée,4,4,0,2,2,17,0,2,2,2,...,0,0,1,2,0,10,17,8,55,2
Infractions au droit de l'urbanisme et de la construction,3,2,2,0,1,169,0,0,0,2,...,0,1,5,1,0,9,4,10,19,3
Fraudes fiscales,1,3,0,0,0,20,0,0,0,1,...,0,1,1,2,0,4,23,3,9,4


### Adding the Corse island

In [8]:
dfSumCorse=dfSum.copy()

In [9]:
dfSumCorse.columns

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '20', '21', '22', '23', '24', '25', '26',
       '27', '28', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39',
       '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51',
       '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63',
       '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75',
       '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87',
       '88', '89', '90', '91', '92', '93', '94', '95'],
      dtype='object')

In [10]:
res=[float(ele) for ele in NP_2012.columns]

In [11]:
A=[1000<=ele<2000 for ele in res]
B=[2000<=ele<3000 for ele in res]

I set arbitrarly 991 and 992 for the two departements of the Corse. 

In [12]:
dfSumCorse['991']=NP_2012IndexC.iloc[:,A].sum(axis=1)
dfSumCorse['992']=NP_2012IndexC.iloc[:,B].sum(axis=1)

In [13]:
dfSumCorse

,1,2,3,4,5,6,7,8,9,10,...,88,89,90,91,92,93,94,95,991,992
crimeType,,,,,,,,,,,,,,,,,,,,,
Règlements de compte entre malfaireurs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,3,0,0,0,0
Homicides pour voler et à l'occasion de vols,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Homicides pour d'autres motifs,1,2,2,0,0,22,0,0,0,0,...,1,0,0,4,15,16,4,16,1,7
Tentatives d'homicides pour voler et à l'occasion de vols,0,1,0,0,0,5,0,0,0,0,...,0,0,0,0,8,4,1,0,0,0
Tentatives homicides pour d'autres motifs,0,5,2,0,0,12,0,0,0,0,...,1,0,3,39,44,64,36,26,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Infractions à l'exercice d'une profession règlementée,4,4,0,2,2,17,0,2,2,2,...,1,2,0,10,17,8,55,2,7,2
Infractions au droit de l'urbanisme et de la construction,3,2,2,0,1,169,0,0,0,2,...,5,1,0,9,4,10,19,3,1,0
Fraudes fiscales,1,3,0,0,0,20,0,0,0,1,...,1,2,0,4,23,3,9,4,0,0


## Generalization

We will :
* create a sheet with the sum of crimes for each year,
* create one sheet a year with all departements.

### Opening every file and set a correct index

In [14]:
def read_file(i):
    return pd.read_excel("crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx",sheet_name=i).rename({"Départements":"crimeType"},axis=1).set_index("crimeType")

In [15]:
NP_2012=read_file(1)
NP_2013=read_file(2)
NP_2014=read_file(3)
NP_2015=read_file(4)
NP_2016=read_file(5)
NP_2017=read_file(6)
NP_2018=read_file(7)
NP_2019=read_file(8)
NP_2020=read_file(9)

In [16]:
#For the local police
LP_2012=read_file(10)
LP_2013=read_file(11)
LP_2014=read_file(12)
LP_2015=read_file(13)
LP_2016=read_file(14)
LP_2017=read_file(15)
LP_2018=read_file(16)
LP_2019=read_file(17)
LP_2020=read_file(18)

### Dataframe with the number of crime by year

In [17]:
dfByYear=pd.DataFrame()

In [18]:
dfByYear["2012"]=NP_2012.sum(axis=1)
dfByYear["2013"]=NP_2013.sum(axis=1)
dfByYear["2014"]=NP_2014.sum(axis=1)
dfByYear["2015"]=NP_2015.sum(axis=1)
dfByYear["2016"]=NP_2016.sum(axis=1)
dfByYear["2017"]=NP_2017.sum(axis=1)
dfByYear["2018"]=NP_2018.sum(axis=1)
dfByYear["2019"]=NP_2019.sum(axis=1)
dfByYear["2020"]=NP_2020.sum(axis=1)

In [19]:
dfByYear

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
Règlements de compte entre malfaireurs,50,34,45,42,59,84,93,71,82
Homicides pour voler et à l'occasion de vols,23,14,15,9,22,19,23,19,24
Homicides pour d'autres motifs,377,387,399,412,465,459,424,514,496
Tentatives d'homicides pour voler et à l'occasion de vols,47,68,43,67,62,74,76,73,62
Tentatives homicides pour d'autres motifs,1035,912,1055,1430,1638,1719,1820,2007,2121
...,...,...,...,...,...,...,...,...,...
Infractions à l'exercice d'une profession règlementée,624,601,562,1199,1196,970,1320,1292,760
Infractions au droit de l'urbanisme et de la construction,899,833,999,1263,1335,1247,1248,1120,896
Fraudes fiscales,617,524,549,341,384,385,377,412,462


### Group by departements for each year

In [20]:
def dfGroupByDep(df):

    data=df.copy()

    columns=data.columns
    lenDF=len(columns)

    dfOneYearAllDeps=pd.DataFrame(index=data.index)
    dfOneDep=pd.DataFrame(index=data.index)

    countDep=1 #counter for the departements 
    countLines=0

    A=False
    B=False

    for index, value in enumerate(columns):

        if value=='30':
            countDep=30

        if value in ['2A','2A.1','2A.2','2A.3','2A.4']: #first case of Corse

                newCol=1000+index
                data[str(newCol)]= data.iloc[:,index] #adding at the end a column

                data=data.drop(str(value),axis=1)
                dfOneDep=pd.DataFrame(index=data.index)

                if columns[index+1]=='2B':
                    A=True

        elif value in ['2B','2B.1','2B.2','2B.3','2B.4']: #second case of Corse
        
                newCol=2000+index
                data[str(newCol)]= data.iloc[:,index] #adding at the end a column
                data=data.drop(str(value),axis=1)

                dfOneDep=pd.DataFrame(index=data.index)

                if columns[index+1]=='30':
                    B=True


        elif ( countDep<=float(value) )  and ( float(value) < countDep+1 ) :

            dfOneDep[str(index)]=data.iloc[:,index]

            try:
                if float(columns[index+1])==21 and countDep+1==20:
                    countDep+=1
            except:
                continue

            if columns[index+1]=='2A':

                countDep=29
                dfOneYearAllDeps[str(countDep)]=dfOneDep.sum(axis=1)  

                dfOneDep=pd.DataFrame(index=data.index)

            if columns[index+1]!='2A':
                if float(columns[index+1])==countDep+1 or float(columns[index+1])==971 and float(value)!=1 :

                    dfOneYearAllDeps[str(countDep)]=dfOneDep.sum(axis=1)
                    countDep+=1

                    dfOneDep=pd.DataFrame(index=data.index)            

    res=[float(ele) for ele in data.columns]

    A=[1000<=ele<2000 for ele in res]
    B=[2000<=ele<3000 for ele in res]

    dfOneYearAllDeps['991']=data.iloc[:,A].sum(axis=1)
    dfOneYearAllDeps['992']=data.iloc[:,B].sum(axis=1)


    return dfOneYearAllDeps

In [21]:
dfByDep2012=dfGroupByDep(NP_2012)
dfByDep2013=dfGroupByDep(NP_2013)
dfByDep2014=dfGroupByDep(NP_2014)
dfByDep2015=dfGroupByDep(NP_2015)
dfByDep2016=dfGroupByDep(NP_2016)
dfByDep2017=dfGroupByDep(NP_2017)
dfByDep2018=dfGroupByDep(NP_2018)
dfByDep2019=dfGroupByDep(NP_2019)
dfByDep2020=dfGroupByDep(NP_2020)

In [47]:
dfByYear

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
Règlements de compte entre malfaireurs,50,34,45,42,59,84,93,71,82
Homicides pour voler et à l'occasion de vols,23,14,15,9,22,19,23,19,24
Homicides pour d'autres motifs,377,387,399,412,465,459,424,514,496
Tentatives d'homicides pour voler et à l'occasion de vols,47,68,43,67,62,74,76,73,62
Tentatives homicides pour d'autres motifs,1035,912,1055,1430,1638,1719,1820,2007,2121
...,...,...,...,...,...,...,...,...,...
Infractions à l'exercice d'une profession règlementée,624,601,562,1199,1196,970,1320,1292,760
Infractions au droit de l'urbanisme et de la construction,899,833,999,1263,1335,1247,1248,1120,896
Fraudes fiscales,617,524,549,341,384,385,377,412,462


## Group by type of crime

### General considerations

We have 107 rows ; we will group by types of crimes:

* related to homicide,
* threats,
* robbery, 
* physical violence,
* drug, alcool, health, food, 
* public degradations,
* fraud, administrative, or related to money.

### Related to homicide

In [56]:
dfByYear.iloc[:7,:]

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
Règlements de compte entre malfaireurs,50,34,45,42,59,84,93,71,82
Homicides pour voler et à l'occasion de vols,23,14,15,9,22,19,23,19,24
Homicides pour d'autres motifs,377,387,399,412,465,459,424,514,496
Tentatives d'homicides pour voler et à l'occasion de vols,47,68,43,67,62,74,76,73,62
Tentatives homicides pour d'autres motifs,1035,912,1055,1430,1638,1719,1820,2007,2121
Coups et blessures volontaires suivis de mort,93,60,87,79,113,88,112,120,100
Autres coups et blessures volontaires criminels ou correctionnels,141918,141464,141757,154479,157782,162087,173174,185316,181981


In [78]:
dfByYear.iloc[50:51,:]

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
Homicides commis contre enfants de moins de 15 ans,24,27,26,41,22,24,26,35,28


### Moral violence (threats)

In [62]:
dfByYear.iloc[10:13,:]

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
Menaces ou chantages pour extorsion de fonds,7458,7411,8580,10103,9903,9602,9766,11201,10563
Menaces ou chantages dans un autre but,59947,61672,63186,72091,72975,72615,77181,84081,81671
Atteintes à la dignité et à la personnalité,19245,19645,24725,28275,27016,27376,30069,31396,29190


### Robbery

In [68]:
dfByYear.iloc[14:44,:]

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
Vols à main armée contre des établissements financiers,188,119,96,101,79,64,51,41,44
Vols à main armée contre des éts industriels ou commerciaux,2468,2530,1960,1709,1246,993,731,708,576
Vols à main armée contre des entreprises de transports de fonds,41,24,26,29,16,17,15,11,8
Vols à main armée contre des particuliers à leur domicile,385,484,457,596,433,395,342,289,301
Autres vols à main armée,1333,1254,1326,1434,1218,1228,1135,1134,1000
"Vols avec armes blanches contre des établissements financiers,commerciaux ou industriels",786,807,591,514,539,500,470,469,352
Vols avec armes blanches contre des particuliers à leur domicile,445,540,436,406,438,442,441,438,410
Autres vols avec armes blanches,6234,6548,5408,4360,4621,4104,3812,3889,3825
"Vols violents sans arme contre des établissements financiers,commerciaux ou industriels",555,632,735,845,831,733,742,630,531


### Physical violence

In [59]:
dfByYear.iloc[7:10,:]

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
Prises d'otages à l'occasion de vols,7,11,16,12,11,14,46,64,53
Prises d'otages dans un autre but,12,15,19,12,19,26,65,98,116
Sequestrations,1655,1662,2373,3300,3231,3195,3226,3166,3041


In [63]:
dfByYear.iloc[13:14,:]

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
Violations de domicile,4637,4840,6284,7667,7827,8400,9363,10125,10859


In [76]:
dfByYear.iloc[44:50,:]

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
Proxénétisme,362,373,317,240,289,319,379,372,359
Viols sur des majeur(e)s,2996,3038,3535,3857,4142,5552,6448,7451,8006
Viols sur des mineur(e)s,2884,2851,2901,3105,3124,4140,4730,5517,5954
Harcèlements sexuels et autres agressions sexuelles contre des majeur(e)s,3674,3822,4429,5302,5810,7181,8898,9381,8571
Harcèlements sexuels et autres agressions sexuelles contre des mineur(e)s,4758,5185,5558,5566,6547,7805,8968,9342,8799
Atteintes sexuelles,9172,8286,6904,6391,6305,7346,8236,8569,8284


In [82]:
dfByYear.iloc[51:54,:]

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
"Violences, mauvais traitements et abandons d'enfants.",14604,15606,22097,26755,28472,30701,33003,35764,31461
Délits au sujet de la garde des mineurs,11641,10859,11454,10044,10977,10918,10626,10079,9822
Non versement de pension alimentaire,6578,6425,6173,5132,5444,5087,4841,4113,3133


In [102]:
dfByYear.iloc[77:80,:]

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
"Destructions, cruautés et autres délits envers les animaux",1085,895,1154,1328,1406,1513,1490,1509,1558
Atteintes à l'environnement,429,430,431,434,469,441,505,582,497
Chasse et pêche,84,84,52,42,63,57,74,86,58


### Drug, alcool, health, food, 

In [87]:
dfByYear.iloc[54:61,:]

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
Trafic et revente sans usage de stupéfiants,5116,5603,5907,6482,7325,9054,10396,11495,10527
Usage-revente de stupéfiants,9007,9784,11418,13699,15234,17165,18322,18924,16516
Usage de stupéfiants,122094,123949,125523,125928,121621,124158,122697,113920,97896
Autres infractions à la législation sur les stupéfiants,3022,2208,1426,523,384,388,632,609,639
Délits de débits de boissons et infraction à la règlementation sur l'alcool et le tabac,599,589,830,1131,1095,1187,1208,1511,1289
Fraudes alimentaires et infractions à l'hygiène,233,232,131,64,70,64,65,70,69
Autres délits contre santé publique et la réglementation des professions médicales,241,171,252,293,256,242,298,282,4989


### Public degradations

In [92]:
dfByYear.iloc[61:68,:]

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
Incendies volontaires de biens publics,3079,3070,3443,3794,3738,4329,4403,4440,3578
Incendies volontaires de biens privés,27142,24061,23545,23652,22483,23804,25009,24373,20518
Attentats à l'explosif contre des biens publics,11,23,26,28,21,28,32,46,51
Attentats à l'explosif contre des biens privés,86,43,35,82,71,72,129,124,137
Autres destructions er dégradations de biens publics,21822,22053,22200,20575,19202,19398,20088,20316,16011
Autres destructions er dégradations de biens privés,105206,102779,98593,94192,92159,88736,92892,97000,83212
Destructions et dégradations de véhicules privés,133031,124492,91629,73457,67145,65373,64052,62752,51364


### Fraud, administrative, or related to money

In [98]:
dfByYear.iloc[68:77,:]

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
Infractions aux conditions générales d'entrée et de séjour des étrangers,79935,21504,18671,11803,4867,3201,3723,3580,3336
"Aide à l'entrée, à la circulation et au séjour des étrangers",4698,4232,4493,4261,4142,4154,4390,4452,3773
Autres infractions à la police des étrangers,3957,3826,3244,2459,2413,2869,4339,3825,3632
Outrages à dépositaires autorité,23231,21680,20235,20401,19730,20115,21798,22743,21590
Violences à dépositaires autorité,24961,24895,25059,26036,25450,26275,29494,30611,27171
Port ou détention armes prohibées,28079,27363,25670,26891,26237,25933,27900,28169,24228
Atteintes aux intérêts fondamentaux de la Nation,148,138,150,297,545,351,298,307,346
Délits des courses et des jeux,366,291,262,143,155,193,134,120,69
Délits interdiction de séjour et de paraître,181,159,368,537,729,910,1080,1374,1504


In [106]:
dfByYear.iloc[80:,:]

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
Faux documents d'identité,6406,6035,5788,6110,5706,6283,6292,6688,5255
Faux documents concernant la circulation des véhicules,2947,3061,2642,2610,2664,2653,2022,1885,2257
Autres faux documents administratifs,2719,2791,2406,2489,2454,2364,2232,2165,1668
Faux en écriture publique et authentique,1390,1424,876,494,366,358,360,375,356
Autres faux en écriture,4195,3959,5856,3467,4066,4209,4395,4520,3826
Fausse monnaie,1057,1397,1844,2441,1888,1767,1296,1630,1435
Contrefaçons et fraudes industrielles et commerciales,784,602,642,615,633,1390,1212,1623,762
Contrefaçons littéraires et artistique,102,133,114,116,206,214,270,226,97
Falsification et usages de chèques volés,42365,38943,42952,48131,52889,54146,48890,45994,38536


## Groupby for every dataset

In [169]:
def groupByType(data):
    df=pd.DataFrame()
    df["Related to homicide"]=pd.Series(data.iloc[:7,:].append(data.iloc[50:51,:]).sum(axis=0))
    df["Moral violence"]=pd.Series(data.iloc[10:13,:].sum(axis=0))
    df["Robbery"]=pd.Series(data.iloc[14:44,:].sum(axis=0))
    df["Physical violence"]=pd.Series(data.iloc[7:10,:].append(data.iloc[13:14,:]).append(data.iloc[44:50,:]).append(data.iloc[51:54,:]).append(data.iloc[77:80,:]).sum(axis=0))
    df["Drug, Alcohol, Health"]=pd.Series(data.iloc[54:61,:].sum(axis=0))
    df["Public degradations"]=pd.Series(data.iloc[61:68,:].sum(axis=0))
    df["Public degradations"]=pd.Series(data.iloc[68:77,:].append(data.iloc[80:,:]).sum(axis=0))

    if data.equals(dfByYear):
        df.index.names=['year']
    else:
        df.index.names=['numDep']
        
    return df

In [171]:
dfByYearGroupByType=groupByType(dfByYear)

In [172]:
dfByYearGroupByType

,Related to homicide,Moral violence,Robbery,Physical violence,"Drug, Alcohol, Health",Public degradations
year,,,,,,
2012,143567,86650,1223643,64578,140312,449367
2013,142966,88728,1273604,64382,142536,400400
2014,143427,96491,1294530,73697,145487,413335
2015,156559,110469,1266334,79187,148120,434679
2016,160163,109894,1242088,84136,145985,434531
2017,164554,109593,1299067,92695,152258,443607
2018,175748,117016,1263021,100898,153618,452517
2019,188155,126678,1277586,106218,146811,485352
2020,184894,121424,1024802,100571,131925,447989


In [173]:
dfGroupByType2012=groupByType(dfByDep2012)
dfGroupByType2013=groupByType(dfByDep2013)
dfGroupByType2014=groupByType(dfByDep2014)
dfGroupByType2015=groupByType(dfByDep2015)
dfGroupByType2016=groupByType(dfByDep2016)
dfGroupByType2017=groupByType(dfByDep2017)
dfGroupByType2018=groupByType(dfByDep2018)
dfGroupByType2019=groupByType(dfByDep2019)
dfGroupByType2020=groupByType(dfByDep2020)

Let's store these datasets for data vizualisation:

In [176]:
dfByYearGroupByType.to_csv('Notebooks/dataVizualisation/crimeInFrance2012_2020.csv')

In [177]:
dfByDep2012.to_csv('/Users/oscarmeurer/iCloud Drive (archive) - 1/Documentss/Pour Disque Dur (last 3 decembre 2021)/DataScience/Projects for public git/Crimes/Notebooks/dataVizualisation/crimeInFranceByDep2012.csv')
dfByDep2013.to_csv('/Users/oscarmeurer/iCloud Drive (archive) - 1/Documentss/Pour Disque Dur (last 3 decembre 2021)/DataScience/Projects for public git/Crimes/Notebooks/dataVizualisation/crimeInFranceByDep2013.csv')
dfByDep2014.to_csv('/Users/oscarmeurer/iCloud Drive (archive) - 1/Documentss/Pour Disque Dur (last 3 decembre 2021)/DataScience/Projects for public git/Crimes/Notebooks/dataVizualisation/crimeInFranceByDep2014.csv')
dfByDep2015.to_csv('/Users/oscarmeurer/iCloud Drive (archive) - 1/Documentss/Pour Disque Dur (last 3 decembre 2021)/DataScience/Projects for public git/Crimes/Notebooks/dataVizualisation/crimeInFranceByDep2015.csv')
dfByDep2016.to_csv('/Users/oscarmeurer/iCloud Drive (archive) - 1/Documentss/Pour Disque Dur (last 3 decembre 2021)/DataScience/Projects for public git/Crimes/Notebooks/dataVizualisation/crimeInFranceByDep2016.csv')
dfByDep2017.to_csv('/Users/oscarmeurer/iCloud Drive (archive) - 1/Documentss/Pour Disque Dur (last 3 decembre 2021)/DataScience/Projects for public git/Crimes/Notebooks/dataVizualisation/crimeInFranceByDep2017.csv')
dfByDep2018.to_csv('/Users/oscarmeurer/iCloud Drive (archive) - 1/Documentss/Pour Disque Dur (last 3 decembre 2021)/DataScience/Projects for public git/Crimes/Notebooks/dataVizualisation/crimeInFranceByDep2018.csv')
dfByDep2019.to_csv('/Users/oscarmeurer/iCloud Drive (archive) - 1/Documentss/Pour Disque Dur (last 3 decembre 2021)/DataScience/Projects for public git/Crimes/Notebooks/dataVizualisation/crimeInFranceByDep2019.csv')
dfByDep2020.to_csv('/Users/oscarmeurer/iCloud Drive (archive) - 1/Documentss/Pour Disque Dur (last 3 decembre 2021)/DataScience/Projects for public git/Crimes/Notebooks/dataVizualisation/crimeInFranceByDep2020.csv')